In [ ]:
import os
import datetime
import time
import decimal

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import random
from binance.client import Client
from binance.exceptions import BinanceAPIException

pd.options.display.float_format = '{:.0f}'.format

# To get api key, https://testnet.binance.vision/
api_key = ""
api_secret = ""


In [2]:
def low(df, curr, p):
    return min(df['Low'][df.index[range(curr - p + 1, curr + 1)]])

def high(df, curr, p):
    return max(df['High'][df.index[range(curr - p + 1, curr + 1)]])

In [3]:
def tri(df, curr):
    high = df['High'][curr]
    low = df['Low'][curr]
    close = df['Close'][curr - 1]
    return max((high - low), 
               abs(high - close),
               abs(low - close))

def trip(df, curr, p):
    return sum([tri(df, i) for i in range(curr - p + 1, curr + 1)])

def upmove(df, curr):
    return df['High'][curr] - df['High'][curr - 1]

def downmove(df, curr):
    return df['Low'][curr - 1] - df['Low'][curr]

def dmplus(df, curr):
    return max(0, upmove(df, curr))

def dmminus(df, curr):
    return max(0, downmove(df, curr))

def dmpplus(df, curr, p):
    return sum([dmplus(df, i) for i in range(curr - p + 1, curr + 1)])

def dmpminus(df, curr, p):
    return sum([dmminus(df, i) for i in range(curr - p + 1, curr + 1)])

def diplus(df, curr, p):
    return dmpplus(df, curr, p)/trip(df, curr, p)

def diminus(df, curr, p):
    return dmpminus(df, curr, p)/trip(df, curr, p)

def dx(df, curr, p):
#     print(df.tail(3))
#     print(curr, p)
    dp = diplus(df, curr, p)
    dm = diminus(df, curr, p)
    dmdf = dp - dm
    dpdf = dp + dm
    DX = abs(dmdf)/dpdf
    return DX, dp, dm

def dxprime(DX, dp, dm, curr):
    return 2*DX[-1]-DX[-2], 2*dp[-1]-dp[-2], 2*dm[-1]-dm[-2]

def dxpp(DXp, dpp, dmp, curr, p):
    DXpp = sum(DXp[curr - p + 1: curr + 1])/p
    dppp = sum(dpp[curr - p + 1: curr + 1])/p
    dmpp = sum(dmp[curr - p + 1: curr + 1])/p
    return DXpp, dppp, dmpp
# def dxpp(DXp, dpp, dmp, curr, p):
#     DXpp = sum([DXp[i]*((i - (curr - p + 1))) for i in range(curr - p + 1, curr + 1)])/((p**2)/2)
#     dppp = sum([dpp[i]*((i - (curr - p + 1))) for i in range(curr - p + 1, curr + 1)])/((p**2)/2)
#     dmpp = sum([dmp[i]*((i - (curr - p + 1))) for i in range(curr - p + 1, curr + 1)])/((p**2)/2)
#     return DXpp, dppp, dmpp

In [4]:
def slicer(start_timestamp, curr_timestamp, step):
    curr_slice = curr_timestamp - (curr_timestamp - start_timestamp)%(step*30*60000)
    return curr_slice

In [5]:
def truncate(number, decimals=0):
    """
    Returns a value truncated to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer.")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more.")
    elif decimals == 0:
        return math.trunc(number)

    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor

In [6]:
#buy a certain amount BTC
#Now with TAXES YEAH
def buy_t(acc_usdt, acc_btc, price, buy_amount):
    acc_usdt -= (buy_amount)*price
    acc_btc += buy_amount
    return acc_usdt, acc_btc

#sell a certain amount BTC
#Now with TAXES YEAH
def sell_t(acc_usdt, acc_btc, price, sell_amount):
    acc_usdt += (sell_amount)*price
    acc_btc -= sell_amount
    return acc_usdt, acc_btc

In [7]:
#buy an amount of BTC with USDT
def buy_b(acc_usdt, acc_btc, price, buy_amount):
    global client_t
    try:
        return client_t.create_order(symbol = "BTCUSDT", side='BUY', type='MARKET', quantity = buy_amount)
    except BinanceAPIException as e:
        print(e)
        return {'status':'FAILED'}

#sell an amount of BTC with USDT
def sell_b(acc_usdt, acc_btc, price, sell_amount):
    global client_t
    try:
        return client_t.create_order(symbol = "BTCUSDT", side='SELL', type='MARKET', quantity = sell_amount)
    except BinanceAPIException as e:
        print(e)
        return {'status':'FAILED'}

In [8]:
def buy(price, weight, acc_usdt, acc_btc):
#     global acc_usdt
#     global acc_btc
    
    out = "Trying to buy..."
    quantity = truncate((acc_usdt/price)*0.995, 3)
#     quantity = ((int((acc_usdt/price)*10**6)-1)/10**6)
    if quantity > 0.0001:
        if(acc_usdt >= quantity*price):
            if real:
                buy_b(acc_usdt, acc_btc, price, quantity)
            else:
                acc_usdt, acc_btc = buy_t(acc_usdt, acc_btc, price, quantity)
            out = out + "Bought " + str(quantity) + "BTC at " + str(price)
            trade = True
            trade_type = "BUY"
        else:
            out = out + "No enough funds : " + str(acc_usdt) + " < " + str(buy_amount*price)
            trade = False
            trade_type = "NONE"
            print("BUY ERROR")
            print(out)
    else:
        out = out + "Trade too small : " + str(quantity)
        trade = False
        trade_type = "NONE"
#     print(out)
    return out, trade_type, trade, acc_usdt, acc_btc

def sell(price, weight, acc_usdt, acc_btc):
#     global acc_usdt
#     global acc_btc
    
    out = "Trying to sell..."
    quantity = truncate(acc_btc, 3)
#     quantity = ((int(acc_btc*10**6)-1)/10**6)
    if quantity > 0.0001:
        if(acc_btc >= (quantity * 0.9999)):
            if real:
                sell_b(acc_usdt, acc_btc, price, quantity)
            else:
                acc_usdt, acc_btc = sell_t(acc_usdt, acc_btc, price, quantity)
            out = out + "Sold " + str(quantity) + "BTC at " + str(price)
            trade = True
            trade_type = "SELL"
        else:
            out = out + "No enough funds : " + str(acc_btc) + " < " + str(quantity)
            trade = False
            trade_type = "NONE"
            print("SELL ERROR")
            print(out)
    else:
        out = out + "Trade too small : " + str(quantity)
        trade = False
        trade_type = "NONE"
#     print(out)
    return out, trade_type, trade, acc_usdt, acc_btc

In [9]:
def get_30m_kline(n):
    
    global client_r
    
    # request historical candle (or klines) data
    bars = []
    while True:
        try:
            t = int(time.time())
            bars = client_r.get_historical_klines('BTCUSDT', '30m', (t - cycle*n)*1000)
            break
        except Exception as e:
            print(e)
            print('Error occured at %s.' % (datetime.datetime.now()))
            time.sleep(60)
            client_r = connect_realnet()
    
    # del useless data
    for line in bars:
        del line[5:]

    # put in dataframe
    bdf = pd.DataFrame(bars, columns=['Open Time', 'Open', 'High', 'Low', 'Close'])
    bdf = bdf.astype(float)
    
    return bdf

def get_4h_kline(n):
    
    global client_r
    
    # request historical candle (or klines) data
    bars = []
    while True:
        try:
            t = int(time.time())
            bars = client_r.get_historical_klines('BTCUSDT', '4h', (t - cycle*step*n)*1000)
            break
        except Exception as e:
            print(e)
            print('Error occured at %s.' % (datetime.datetime.now()))
            time.sleep(60)
            client_r = connect_realnet()
    
    # del useless data
    for line in bars:
        del line[5:]

    # put in dataframe
    ldf = pd.DataFrame(bars, columns=['Open Time', 'Open', 'High', 'Low', 'Close'])
    ldf = ldf.astype(float)
    
    return ldf

In [10]:
def precalc_p_short(p):
    global DX, dp, dm
    global DXp, dpp, dmp
    global DXpp, dppp, dmpp
    
    dx_res = [dx(bdf[curr - p:curr + 1], 
                 curr, 
                 p) 
              for curr in range(p, len(bdf))]
    DX, dp, dm = map(list, zip(*dx_res))
    pad = [0 for i in range(p)]
    DX = pad + DX
    dp = pad + dp
    dm = pad + dm

def precalc_p_short_prime(p):
    global DX, dp, dm
    global DXp, dpp, dmp
    global DXpp, dppp, dmpp
    
    dxprime_res = [dxprime(DX[curr - 3:curr + 1], 
                           dp[curr - 3:curr + 1], 
                           dm[curr - 3:curr + 1], 
                           curr) 
                   for curr in range(p+1, len(bdf))]
    DXp, dpp, dmp = map(list, zip(*dxprime_res))
    pad = [0 for i in range(p+1)]
    DXp = pad + DXp
    dpp = pad + dpp
    dmp = pad + dmp

def precalc_q_short(p, q):
    
    global df
    global DXpp, dppp, dmpp
    global pdp, mdp, pdm, mdm
    
    dxpp_res = [dxpp(DXp, dpp, dmp, curr, q) for curr in range(p+q+1, len(bdf))]
    DXpp, dppp, dmpp = map(list, zip(*dxpp_res))
    pad = [0 for i in range(p+q+1)]
    DXpp = pad + DXpp
    dppp = pad + dppp
    dmpp = pad + dmpp
    
    pdp = pad + [max(dpp[curr], dppp[curr]) for curr in range(p+q+1, len(bdf))]
    mdp = pad + [min(dpp[curr], dppp[curr]) for curr in range(p+q+1, len(bdf))]
    pdm = pad + [max(dmp[curr], dmpp[curr]) for curr in range(p+q+1, len(bdf))]
    mdm = pad + [min(dmp[curr], dmpp[curr]) for curr in range(p+q+1, len(bdf))]

def make_candle(curr, p):
    global bdf
    global ldf
    
    curr_open_time = bdf['Open Time'][bdf.index[curr]]

    curr_l = ldf.index[ldf['Open Time'] <= curr_open_time][-1]

    curr_large_time = ldf['Open Time'][ldf.index[curr_l]]

    tmp = bdf[bdf['Open Time'] >= curr_large_time]
    tmp2 = tmp[tmp['Open Time'] <= curr_open_time]

    op = ldf['Open'][ldf.index[curr_l]]
    hi = max(tmp2['High'])
    lo = min(tmp2['Low'])
    cl = bdf['Close'][bdf.index[curr]]

    last = pd.DataFrame([[curr_large_time, op, hi, lo, cl]], columns = ["Open Time", "Open", "High", "Low", "Close"])
    last.rename(index={0:curr_l},inplace=True)

    DXt, dpt, dmt = dx(pd.concat([ldf[curr_l - p:curr_l], last]), curr_l, p)
    return DXt, dpt, dmt
    
def precalc_p_long(p):
    
    global bdf
    global ldf
    global DXl, dpl, dml
    global DXpl, dppl, dmpl
    global DXppl, dpppl, dmppl
    global step
    
    global curr, curr_open_time, curr_large_time
    global curr_l
    
    t1 = time.time()
    
    DXl = []
    dpl = []
    dml = []
    
    dx_res = [make_candle(curr, p)
              for curr in range(p*step, len(bdf))]
    DXl, dpl, dml = map(list, zip(*(dx_res)))
    
    t2 = time.time()
    pad = [0 for i in range(p*step)]
    DXl = pad + DXl
    dpl = pad + dpl
    dml = pad + dml

def precalc_p_long_prime(p):
    global df
    global bdf
    global ldf
    global DXl, dpl, dml
    global DXpl, dppl, dmpl
    global DXppl, dpppl, dmppl
    global step
    t1 = time.time()
    dxprime_res = [dxprime(DXl[curr - 3:curr + 1], 
                           dpl[curr - 3:curr + 1], 
                           dml[curr - 3:curr + 1], 
                           curr) 
                   for curr in range((p+1)*step, len(bdf))]
    DXpl, dppl, dmpl = map(list, zip(*dxprime_res))
    t2 = time.time()
    pad = [0 for i in range((p+1)*step)]
    DXpl = pad + DXpl
    dppl = pad + dppl
    dmpl = pad + dmpl

def precalc_q_long(p, q):
    
    global df
    global bdf
    global ldf
    global DXppl, dpppl, dmppl
    global pdpl, mdpl, pdml, mdml
    
    t1 = time.time()
    dxpp_res = [dxpp(DXpl, dppl, dmpl, curr, q) for curr in range((p+q+1)*step, len(bdf))]
    DXppl, dpppl, dmppl = map(list, zip(*dxpp_res))
    t2 = time.time()
    pad = [0 for i in range((p+q+1)*step)]
    DXppl = pad + DXppl
    dpppl = pad + dpppl
    dmppl = pad + dmppl
    
    pdpl = [max(dppl[curr], dpppl[curr]) for curr in range(len(bdf))]
    mdpl = [min(dppl[curr], dpppl[curr]) for curr in range(len(bdf))]
    pdml = [max(dmpl[curr], dmppl[curr]) for curr in range(len(bdf))]
    mdml = [min(dmpl[curr], dmppl[curr]) for curr in range(len(bdf))]

In [11]:
# bdf = get_30m_kline(16)[:-1]
# ldf = get_4h_kline(2)

In [12]:
# bdf

In [13]:
# ldf

In [14]:
# tmp2

In [15]:
# curr = -1

# curr_open_time = bdf['Open Time'][bdf.index[curr]]

# curr_l = ldf.index[ldf['Open Time'] <= curr_open_time][-1]

# curr_large_time = ldf['Open Time'][ldf.index[curr_l]]

# tmp = bdf[bdf['Open Time'] >= curr_large_time]
# tmp2 = tmp[tmp['Open Time'] <= curr_open_time]

# op = ldf['Open'][ldf.index[curr_l]]
# hi = max(tmp2['High'])
# lo = min(tmp2['Low'])
# cl = bdf['Close'][bdf.index[curr]]

# last = pd.DataFrame([[curr_large_time, op, hi, lo, cl]], columns = ["Open Time", "Open", "High", "Low", "Close"])
# last.rename(index={0:curr_l},inplace=True)

# last

In [16]:
# pd.concat([ldf[curr_l - p:curr_l], last])

In [17]:
def trade_func(curr, price, weight, acc_usdt, acc_btc, last_trade_type):
    
    trade = True
    trade_type = "NONE"
    
    if (pdpl[curr] > mdml[curr]):
        # BULL
        # BUY
        if(last_trade_type !=  'BUY' and (pdp[curr] > mdm[curr])):
            price = bdf['Close'][bdf.index[curr]]
            out, trade_type, trade, acc_usdt, acc_btc = buy(price, weight, acc_usdt, acc_btc)
        # SELL
        if(last_trade_type != 'SELL' and (pdp[curr] < mdm[curr] and abs(pdp[curr] - mdm[curr]) > 0.02)):
            price = bdf['Close'][bdf.index[curr]]
            out, trade_type, trade, acc_usdt, acc_btc = sell(price, weight, acc_usdt, acc_btc)
    
    elif (mdpl[curr] < pdml[curr]):
        # BEAR
        # BUY
        if(last_trade_type !=  'BUY' and (mdp[curr] > pdm[curr])):
            price = bdf['Close'][bdf.index[curr]]
            out, trade_type, trade, acc_usdt, acc_btc = buy(price, weight, acc_usdt, acc_btc)
        # SELL
        if(last_trade_type != 'SELL' and (mdp[curr] < pdm[curr])):
            price = bdf['Close'][bdf.index[curr]]
            out, trade_type, trade, acc_usdt, acc_btc = sell(price, weight, acc_usdt, acc_btc)
    
    if(trade_type == "NONE"):
        out = "No trade"
        trade = False

    if trade:
        return price, trade_type, acc_usdt, acc_btc
    else:
        return 0, last_trade_type, acc_usdt, acc_btc

def reverse_trade_func(curr, price, weight, acc_usdt, acc_btc, last_trade_type):
    
    trade = True
    trade_type = "NONE"
    
    if (pdpl[curr] > mdml[curr]):
        # BULL
        # BUY
        if(last_trade_type !=  'SELL' and (pdp[curr] > mdm[curr])):
            price = bdf['Close'][bdf.index[curr]]
            out, trade_type, trade, acc_usdt, acc_btc = sell(price, weight, acc_usdt, acc_btc)
        # SELL
        if(last_trade_type != 'BUY' and (pdp[curr] < mdm[curr] and abs(pdp[curr] - mdm[curr]) > 0.02)):
            price = bdf['Close'][bdf.index[curr]]
            out, trade_type, trade, acc_usdt, acc_btc = buy(price, weight, acc_usdt, acc_btc)
    
    elif (mdpl[curr] < pdml[curr]):
        # BEAR
        # BUY
        if(last_trade_type !=  'SELL' and (mdp[curr] > pdm[curr])):
            price = bdf['Close'][bdf.index[curr]]
            out, trade_type, trade, acc_usdt, acc_btc = sell(price, weight, acc_usdt, acc_btc)
        # SELL
        if(last_trade_type != 'BUY' and (mdp[curr] < pdm[curr])):
            price = bdf['Close'][bdf.index[curr]]
            out, trade_type, trade, acc_usdt, acc_btc = buy(price, weight, acc_usdt, acc_btc)
    
    if(trade_type == "NONE"):
        out = "No trade"
        trade = False
    else:
        if(trade_type == "BUY"):
            trade_type = "SELL"
        elif(trade_type == "SELL"):
            trade_type = "BUY"

    if trade:
        return price, trade_type, acc_usdt, acc_btc
    else:
        return 0, last_trade_type, acc_usdt, acc_btc

In [18]:
def init_df(t, cycle):
    global bdf, ldf
    global out_df
    global DX, dp, dm
    global DXp, dpp, dmp
    global DXpp, dppp, dmpp
    global pdp, mdp, pdm, mdm
    global DXl, dpl, dml
    global DXpl, dppl, dmpl
    global DXppl, dpppl, dmppl
    global pdpl, mdpl, pdml, mdml
    global client_r
    
    global pl, ql, p, q
    
    bdf = get_30m_kline(500)[:-1]
    ldf = get_4h_kline(100)
    
    precalc_p_short(p)
    precalc_p_short_prime(p)
    precalc_q_short(p, q)
    
    precalc_p_long(pl)
    precalc_p_long_prime(pl)
    precalc_q_long(pl, ql)
    
    out_df = pd.DataFrame()
    
    client_r = connect_realnet()
    client_t = connect_testnet()
    
    while True:
        try:
            acc_usdt = float(client_t.get_asset_balance(asset='USDT')['free'])
            acc_btc = float(client_t.get_asset_balance(asset='BTC')['free'])
            c_close = bdf['Close'][bdf.index[-1]]
            break
        except Exception as e:
            print("Failed to read states from server during init : ")
            print(e)
            client_r = connect_realnet()
            client_t = connect_testnet()
    
    trade_type = "NONE"
    last_trade_type = "NONE"
    for i in range(len(bdf)):
        price = bdf['Close'][bdf.index[i]]
        ret_price, last_trade_type, acc_usdt, acc_btc = trade_func(i, price, 1, acc_usdt, acc_btc, last_trade_type)
        if ret_price != 0:
            last_trade_price = ret_price
            trade_type = last_trade_type
        else:
            trade_type = "NONE"
        temp_df = pd.DataFrame([[bdf['Open Time'][i], 
                                 bdf['High'][i], 
                                 bdf['Low'][i], 
                                 bdf['Close'][i], 
                                 trade_type, 
                                 acc_usdt,
                                 acc_btc,
                                 acc_usdt + acc_btc*price,
                                 DX[i],
                                 dp[i], 
                                 dm[i],
                                 DXp[i],
                                 dpp[i],
                                 dmp[i],
                                 dppp[i],
                                 dmpp[i],
                                 DXl[i],
                                 dpl[i], 
                                 dml[i],
                                 DXpl[i],
                                 dppl[i],
                                 dmpl[i],
                                 dpppl[i],
                                 dmppl[i],
                                 pdp[i],
                                 mdp[i],
                                 pdm[i],
                                 mdm[i],
                                 pdpl[i],
                                 mdpl[i],
                                 pdml[i],
                                 mdml[i]]],columns = out_columns)
        temp_df.rename(index={0:i},inplace=True)
        out_df = out_df.append(temp_df)
    
    client_r = connect_realnet()
    client_t = connect_testnet()
    
    while True:
        try:
            acc_usdt = float(client_t.get_asset_balance(asset='USDT')['free'])
            acc_btc = float(client_t.get_asset_balance(asset='BTC')['free'])
            break
        except Exception as e:
            print("Failed to read states from server during init : ")
            print(e)
            client_r = connect_realnet()
            client_t = connect_testnet()
    
    out_df.loc[len(out_df)-1, ('Account USDT')] = acc_usdt
    out_df.loc[len(out_df)-1, ('Account BTC')] = acc_btc
    out_df.loc[len(out_df)-1, ('Account Value')] = acc_usdt + acc_btc*out_df.loc[len(out_df)-1, ('Close')]
    
    if acc_usdt > acc_btc*c_close:
        last_trade_type = "SELL"
    else:
        last_trade_type = "BUY"
        
    tmp = out_df.loc[len(out_df)-1, ('Action')]
    
    for i in range(len(out_df)-1, 0, -1):
        
        if out_df.loc[i-1, ('Action')] != out_df.loc[i, ('Action')] and out_df.loc[i, ('Action')] != "NONE":
            if out_df.loc[i, ('Action')] == "SELL":
                out_df.loc[i-1, ('Account USDT')] = 0
                out_df.loc[i-1, ('Account BTC')] = out_df.loc[i, ('Account BTC')] + out_df.loc[i, ('Account USDT')]/out_df.loc[i, ('Close')]
            elif out_df.loc[i, ('Action')] == "BUY":
                out_df.loc[i-1, ('Account USDT')] = out_df.loc[i, ('Account USDT')] + out_df.loc[i, ('Account BTC')]*out_df.loc[i, ('Close')]
                out_df.loc[i-1, ('Account BTC')] = 0
        else:
            out_df.loc[i-1, ('Account USDT')] = out_df.loc[i, ('Account USDT')]
            out_df.loc[i-1, ('Account BTC')] = out_df.loc[i, ('Account BTC')]
            
        out_df.loc[i, ('Account Value')] = out_df.loc[i, ('Account USDT')] + out_df.loc[i, ('Account BTC')]*out_df.loc[i, ('Close')]
        
#     out_df = out_df.iloc[::-1]
    out_df.loc[len(out_df)-1, ('Action')] = tmp
    
    print("Initialisation complete")

In [19]:
# # out_df.rename(index={0:len(out_df)},inplace=True)
# out_df['Account USDT'][12]

In [20]:
def update_df(t, cycle):
    global bdf, ldf
    global out_df
    global DX, dp, dm
    global DXp, dpp, dmp
    global DXpp, dppp, dmpp
    global pdp, mdp, pdm, mdm
    global DXl, dpl, dml
    global DXpl, dppl, dmpl
    global DXppl, dpppl, dmppl
    global pdpl, mdpl, pdml, mdml
    global client_r
    global step
    
    global debug
    
    global pl, ql, p, q
    
    bdf = bdf.append(get_30m_kline(2)[:-1], ignore_index=True)
    
    last_large_time = ldf['Open Time'][ldf.index[-1]]
    
#     print(last_large_time)
    
    new_candle = get_4h_kline(2)
    
#     print(new_candle)
    
    # if new candle is first of 4h period
    if new_candle['Open Time'][new_candle.index[-1]] != last_large_time:
        ldf = ldf.append(new_candle[:-1], ignore_index=True)
    
    ldf = ldf[:-1].append(new_candle[-1:], ignore_index=True)
    
#     print(ldf.tail(3))
    
    curr = len(bdf)-1
    curr_l = len(ldf)-1
    
    DXt, dpt, dmt = dx(bdf, curr, p)
    DX.append(DXt)
    dp.append(dpt)
    dm.append(dmt)
    DXpt, dppt, dmpt = dxprime(DX, dp, dm, curr)
    DXp.append(DXpt)
    dpp.append(dppt)
    dmp.append(dmpt)
    DXppt, dpppt, dmppt = dxpp(DXp, dpp, dmp, curr, q)
    DXpp.append(DXppt)
    dppp.append(dpppt)
    dmpp.append(dmppt)
    
    curr = -1
    pdp = pdp + [max(dpp[curr], dppp[curr])]
    mdp = mdp + [min(dpp[curr], dppp[curr])]
    pdm = pdm + [max(dmp[curr], dmpp[curr])]
    mdm = mdm + [min(dmp[curr], dmpp[curr])]
    
    curr = len(bdf)-1
    curr_l = len(ldf)-1
    
    DXlt, dplt, dmlt = dx(ldf, curr_l, pl)
    DXl.append(DXlt)
    dpl.append(dplt)
    dml.append(dmlt)
    DXplt, dpplt, dmplt = dxprime(DXl, dpl, dml, curr)
    DXpl.append(DXplt)
    dppl.append(dpplt)
    dmpl.append(dmplt)
    DXpplt, dppplt, dmpplt = dxpp(DXpl, dppl, dmpl, curr, ql)
    DXppl.append(DXpplt)
    dpppl.append(dppplt)
    dmppl.append(dmpplt)
    
    debug.append(["ldf", DXlt, dplt, dmlt, DXplt, dpplt, dmplt, DXpplt, dppplt, dmpplt])
    
#     DXlt, dplt, dmlt = dx(ldf.append(new_candle[-1:], ignore_index=True), curr_l, pl)
#     DXl.append(DXlt)
#     dpl.append(dplt)
#     dml.append(dmlt)
#     DXplt, dpplt, dmplt = dxprime(DXl, dpl, dml, curr)
#     DXpl.append(DXplt)
#     dppl.append(dpplt)
#     dmpl.append(dmplt)
#     DXpplt, dppplt, dmpplt = dxpp(DXpl, dppl, dmpl, curr, ql)
#     DXppl.append(DXpplt)
#     dpppl.append(dppplt)
#     dmppl.append(dmpplt)
    
#     debug.append(["can", DXlt, dplt, dmlt, DXplt, dpplt, dmplt, DXpplt, dppplt, dmpplt])
    
#     DXlt, dplt, dmlt = dx(ldf.append(new_candle[-1:], ignore_index=True), curr_l+1, pl)
#     DXl.append(DXlt)
#     dpl.append(dplt)
#     dml.append(dmlt)
#     DXplt, dpplt, dmplt = dxprime(DXl, dpl, dml, curr)
#     DXpl.append(DXplt)
#     dppl.append(dpplt)
#     dmpl.append(dmplt)
#     DXpplt, dppplt, dmpplt = dxpp(DXpl, dppl, dmpl, curr, ql)
#     DXppl.append(DXpplt)
#     dpppl.append(dppplt)
#     dmppl.append(dmpplt)
    
#     debug.append(["can+1", DXlt, dplt, dmlt, DXplt, dpplt, dmplt, DXpplt, dppplt, dmpplt])
    
    curr = -1
    pdpl = pdpl + [max(dppl[curr], dpppl[curr])]
    mdpl = mdpl + [min(dppl[curr], dpppl[curr])]
    pdml = pdml + [max(dmpl[curr], dmppl[curr])]
    mdml = mdml + [min(dmpl[curr], dmppl[curr])]

In [21]:
def trader(curr, p, q, last_trade_price, last_trade_type, acc_usdt, acc_btc):
    global pdp, mdp, pdm, mdm
    global pdpl, mdpl, pdml, mdml
    global step
    
    weight = 1
    
    trade = True
    trade_type = "NONE"
    
#     pdp = max(dpp[curr], dppp[curr])
#     mdp = min(dpp[curr], dppp[curr])
#     pdm = max(dmp[curr], dmpp[curr])
#     mdm = min(dmp[curr], dmpp[curr])
    
#     mdp[curr] > pdm[curr] = easy to sell, hard to buy, bear market
#     pdp[curr] > mdm[curr] = easy to buy, hard to sell, bull market
#     find switch between the two
    
    if (pdpl[curr] > mdml[curr]):
        # BULL
        # BUY
        if(last_trade_type !=  'BUY' and (pdp[curr] > mdm[curr])):
            price = bdf['Close'][bdf.index[curr]]
            out, trade_type, trade, acc_usdt, acc_btc = buy(price, weight, acc_usdt, acc_btc)
        # SELL
        if(last_trade_type != 'SELL' and (pdp[curr] < mdm[curr] and abs(pdp[curr] - mdm[curr]) > 0.02)):
            price = bdf['Close'][bdf.index[curr]]
            out, trade_type, trade, acc_usdt, acc_btc = sell(price, weight, acc_usdt, acc_btc)
    
    elif (mdpl[curr] < pdml[curr]):
        # BEAR
        # BUY
        if(last_trade_type !=  'BUY' and (mdp[curr] > pdm[curr])):
            price = bdf['Close'][bdf.index[curr]]
            out, trade_type, trade, acc_usdt, acc_btc = buy(price, weight, acc_usdt, acc_btc)
        # SELL
        if(last_trade_type != 'SELL' and (mdp[curr] < pdm[curr])):
            price = bdf['Close'][bdf.index[curr]]
            out, trade_type, trade, acc_usdt, acc_btc = sell(price, weight, acc_usdt, acc_btc)
    
    if(trade_type == "NONE"):
        out = "No trade"
        trade = False
    
    print(out)

    if trade:
        return price, trade_type, acc_usdt, acc_btc
    else:
        return 0, last_trade_type, acc_usdt, acc_btc

In [22]:
def connect_realnet():
    # Connect client to realnet api
    client = Client(api_key, api_secret, testnet=False)
    client.API_URL = 'https://api.binance.com/api'
    return client

def connect_testnet():
    if live:
        # Connect client to realnet api
        client = Client(api_key, api_secret, testnet=False)
        client.API_URL = 'https://api.binance.com/api'
        return client
    else:
        # Connect client to testnet api
        client = Client(api_key, api_secret, testnet=True)
        client.API_URL = 'https://testnet.binance.vision/api'
        return client

In [23]:
out_columns = ["Timestamp", 
               "High", 
               "Low", 
               "Close", 
               "Action", 
               "Account USDT", 
               "Account BTC", 
               "Account Value",
               "DX",
               "dp", 
               "dm", 
               "DXp", 
               "dpp", 
               "dmp", 
               "dppp", 
               "dmpp",
               "DXl",
               "dpl", 
               "dml", 
               "DXpl", 
               "dppl", 
               "dmpl", 
               "dpppl", 
               "dmppl",
               "pdp",
               "mdp",
               "pdm",
               "mdm",
               "pdpl",
               "mdpl",
               "pdml",
               "mdml",
              ]

In [24]:
pl = 5
ql = 15
p = 16
q = 32

step = 8

last_trade_type = 'NONE'
last_trade_price = 0

file = "RT_test.csv"

debug = []

In [25]:
cycle = 1800

client_r = connect_realnet()

#---LIVE-TRIGGER-----
live = False
#--------------------

real = False
t = int(time.time())
init_df(t, cycle)
out_df.to_csv(file)

client_r = connect_realnet()
client_t = connect_testnet()
while True:
    try:
        acc_usdt = float(client_t.get_asset_balance(asset='USDT')['free'])
        acc_btc = float(client_t.get_asset_balance(asset='BTC')['free'])
        c_close = bdf['Close'][bdf.index[-1]]
        break
    except Exception as e:
        print("Failed to read states from server during init : ")
        print(e)
        client_r = connect_realnet()
        client_t = connect_testnet()

if acc_usdt > acc_btc*c_close:
    last_trade_type = "SELL"
else:
    last_trade_type = "BUY"

acc_value = acc_usdt + acc_btc*c_close
print(" - State : " + str(last_trade_type))
print(" - Result : " + str(acc_usdt) + " USDT, " + str(acc_btc) + " BTC, " + str(acc_value) + " total USDT")

prev_time = 0
real = True
while True:
    t = int(time.time())
    if t % cycle == 5 and t != prev_time:
        
        prev_time = t

        timestamp = (t - cycle*30)*1000

        client_r = connect_realnet()
        client_t = connect_testnet()
        while True:
            try:
                acc_usdt = float(client_t.get_asset_balance(asset='USDT')['free'])
                acc_btc = float(client_t.get_asset_balance(asset='BTC')['free'])
                break
            except Exception as e:
                print("Failed to read states from server before update : ")
                print(e)
                client_r = connect_realnet()
                client_t = connect_testnet()

        update_df(t, cycle)

        ret_price, last_trade_type, acc_usdt, acc_btc = trader(-1, p, q, last_trade_price, last_trade_type, acc_usdt, acc_btc)
        if ret_price != 0:
            last_trade_price = ret_price
            trade_type = last_trade_type
        else:
            trade_type = "NONE"

        # log everything
        client_r = connect_realnet()
        client_t = connect_testnet()
        while True:
            try:
                acc_usdt = float(client_t.get_asset_balance(asset='USDT')['free'])
                acc_btc = float(client_t.get_asset_balance(asset='BTC')['free'])
                c_close = bdf['Close'][bdf.index[-1]]
                break
            except Exception as e:
                print("Failed to read states from server during log")
                print(e)
                time.sleep(60)
                client_r = connect_realnet()
                client_t = connect_testnet()

        acc_value = acc_usdt + acc_btc*c_close

        temp_df = pd.DataFrame([[bdf['Open Time'][bdf.index[-1]], 
                                 bdf['High'][bdf.index[-1]], 
                                 bdf['Low'][bdf.index[-1]], 
                                 bdf['Close'][bdf.index[-1]], 
                                 trade_type, 
                                 acc_usdt,
                                 acc_btc,
                                 acc_value,
                                 DX[-1],
                                 dp[-1], 
                                 dm[-1],
                                 DXp[-1],
                                 dpp[-1],
                                 dmp[-1],
                                 dppp[-1],
                                 dmpp[-1],
                                 DXl[-1],
                                 dpl[-1], 
                                 dml[-1],
                                 DXpl[-1],
                                 dppl[-1],
                                 dmpl[-1],
                                 dpppl[-1],
                                 dmppl[-1],
                                 pdp[-1],
                                 mdp[-1],
                                 pdm[-1],
                                 mdm[-1],
                                 pdpl[-1],
                                 mdpl[-1],
                                 pdml[-1],
                                 mdml[-1]]], columns = out_columns)
        temp_df.rename(index={0:len(out_df)},inplace=True)
        out_df = out_df.append(temp_df)

        out_df.to_csv(file)

        print(" - Result : " + str(acc_usdt) + " USDT, " + str(acc_btc) + " BTC, " + str(acc_value) + " total USDT")
        print(" - Price  : " + str(c_close) + " USDT")
#         print(debug)

KeyboardInterrupt: 